Oh Crap is an trick taking game played with a standard 52-card deck. In the first round, 1 card is dealt to each player. The trump suit is then determined by revealing the top card of the deck. The number of cards dealt increases by 1 each round, but here I'm only concerned about the first round. I want to determine wether the leading player should guess they will get either 0 or 1 tricks.

Obviously this depends on the card the leading playing is dealt, the number of players (2-7), and the scoring system. In Oh Crap, you get 10 points for correctly guessing 1 trick, 5 points for correctly guessing 0 tricks, and 0 points for any incorrect guess. The winning card in the first round must either be of the leading suit, or trump. This means that all 26 cards of the other 2 suits are equally worthless. The 1 card used to determine trump is known to all players, but it can not be dealt to any of the players. This leaves a list of 25 cards that could possibly win round 1. These 25 cards can be ordered by value. The leading player is garunteed to be dealt one of these 25 cards.

In [1]:
# Theory and Imports

"""
Let x represent the value of the card dealt to the leading player (1 [low card] <= x <= 25 [high card])

Let Po represet (25 - x)/50 (the probability of any 1 opponent having a higher value card than me)

E0 = 5*P(loss)
E0 = 5*(1 - P(win))

E1 = 10*P(win)

2 Player:
P2(win) = 1 - P(loss)
P2(win) = 1 - P(opponent beats me)
P2(win) = 1 - (cards that beat me)/(cards available to opponent)
P2(win) = 1 - (25 - x)/50
P2(win) = 1 - Po

3 Player:
P3(win) = 1 - P(loss)
P3(win) = 1 - P(opponent 1 beats me) - P(opponent 2 beats me) + P(opponent 1 and opponent 2 beat me)
P3(win) = 1 - 2*P(opponent 1 beats me) + P(opponent 1 beats me)^2
P3(win) = 1 - 2*(25 - x)/50 + ((25 - x)/50)^2
P3(win) = 1 - 2*Po + Po^2

4 Player:
P4(win) = 1 - P(loss)
P4(win) = 1 - P(opp1) - P(opp2) - P(opp3) + P(opp1 & opp2) + P(opp2 & opp3) + P(opp1 & opp3) - P(opp1 & opp2 & opp3)
P4(win) = 1 - 3*P(opp1) + 3*P(opp1 & opp2) - P(opp1 & opp2 & opp3)
P4(win) = 1 - 3*P(opp1) + 3*P(opp1)^2 - P(opp1)^3
P4(win) = 1 - 3*Po + 3*Po^2 - Po^3

5 Player:
P5(win) = 1 - P(loss)
P5(win) = 1 - P(1) - P(2) - P(3) - P(4) + P(1,2) + P(1,3) + P(1,4) + P(2,3) + P(2,4) + P(3,4) 
            - P(1,2,3) - P(1,2,4) - P(1,3,4) - P(2,3,4) + P(1,2,3,4)
P5(win) = 1 - 4*P(1) + 6*P(1,2) - 4*P(1,2,3) + P(1,2,3,4)
P5(win) = 1 - 4*Po + 6*Po^2 - 4*Po^3 + Po^4

6 Player:
P6(win) = 1 - 5*Po + 10*Po^2 - 10*Po^3 + 5*Po^4 - Po^5

7 Player:
P7(win) = 1 - 6*Po + 15*Po^2 - 20*Po^3 + 15*Po^4 - 6*Po^5 + Po^6

The coeffients are like Pascal's Triangle, neat.

"""

import numpy as np
import pandas as pd
import random as rm

In [2]:
# Analytical Solution

# based on the theory from above
def probWinning(x, nPlayers):
    assert nPlayers >=2 and nPlayers <= 7
    
    Po = (25 - x)/50
    
    if nPlayers == 2:
        return 1 - Po
    if nPlayers == 3:
        return 1 - 2*Po + Po**2
    if nPlayers == 4:
        return 1 - 3*Po + 3*Po**2 - Po**3
    if nPlayers == 5:
        return 1 - 4*Po + 6*Po**2 - 4*Po**3 + Po**4
    if nPlayers == 6:
        return 1 - 5*Po + 10*Po**2 - 10*Po**3 + 5*Po**4 - Po**5
    if nPlayers == 7:
        return 1 - 6*Po + 15*Po**2 - 20*Po**3 + 15*Po**4 - 6*Po**5 + Po**6

# gets the expectation values for guessing both 1 and 0, for every possible leading card
def getExpectationValues(nPlayers):
    expectationValues = pd.DataFrame(columns=['E0', 'E1'])
    
    for x in range(1, 26):
        Pwin = probWinning(x, nPlayers)
        E0 = 5*(1 - Pwin)
        E1 = 10*Pwin
        expectationValues.loc[x] = [E0, E1]
    
    return expectationValues

for numPlayers in range(2, 8):
    EV = getExpectationValues(nPlayers = numPlayers)
    EV['Guess1'] = EV.apply(lambda x: x.E1 > x.E0, axis=1)
    lowestValue = EV.query('Guess1 == 1').index.min()
    print(f'For {numPlayers} players, 1 trick should be guessed for any card with value {lowestValue}/25 or higher')

For 2 players, 1 trick should be guessed for any card with value 1/25 or higher
For 3 players, 1 trick should be guessed for any card with value 4/25 or higher
For 4 players, 1 trick should be guessed for any card with value 10/25 or higher
For 5 players, 1 trick should be guessed for any card with value 13/25 or higher
For 6 players, 1 trick should be guessed for any card with value 16/25 or higher
For 7 players, 1 trick should be guessed for any card with value 17/25 or higher


In [3]:
# Numerical Solution

# runs a simulated first round of Oh Crap and returns the leading players score
def runGame(leadingCard, nPlayers, numTricksGuessed):
    assert numTricksGuessed in [0, 1]
    assert nPlayers >=2 and nPlayers <= 7
    
    playerValues = []
    for i in range(nPlayers):
        if i == 0:
            playerValues.append(26 - leadingCard)
        else:
            while len(playerValues) == i:
                x = rm.randint(1, 50)
                if x not in playerValues:
                    playerValues.append(x)
                    
    if playerValues[0] == np.array(playerValues).min() and numTricksGuessed == 1:
        return 10
    elif playerValues[0] != np.array(playerValues).min() and numTricksGuessed == 0:
        return 5
    return 0

# simulates a game for every possible leading card, across all possible number of players
def runManyGames(nGames, numTricksGuessed):
    expectationValues = pd.DataFrame(columns=['2P', '3P', '4P', '5P', '6P', '7P'])
    for x in range(1, 26):
        pointsByNumPlayers = []
        for numPlayers in range(2, 8):
            pointSum = 0
            for gameNumber in range(nGames):
                pointSum += runGame(x, numPlayers, numTricksGuessed)
            pointsByNumPlayers.append(pointSum/nGames)
        expectationValues.loc[x] = pointsByNumPlayers
    return expectationValues
    
numGames = 10000
EV_difference = runManyGames(numGames, 1) - runManyGames(numGames, 0)
for i, column in enumerate(EV_difference):
    for j, value in enumerate(EV_difference[column]):
        if float(value) >= 0:
            print(f'For {i+2} players, 1 trick should be guessed for any card with value {j+1}/25 or higher')
            break

For 2 players, 1 trick should be guessed for any card with value 1/25 or higher
For 3 players, 1 trick should be guessed for any card with value 5/25 or higher
For 4 players, 1 trick should be guessed for any card with value 11/25 or higher
For 5 players, 1 trick should be guessed for any card with value 14/25 or higher
For 6 players, 1 trick should be guessed for any card with value 16/25 or higher
For 7 players, 1 trick should be guessed for any card with value 18/25 or higher


Hmm, the analytical solutions don't match numerical ones. I expect there is something wrong with my math in the analytical theory. I'll have to take another look at this sometime to figure out the mismatch. 